2021/10/10

This code is for obtaining radial file base anatomical parameters.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import cv2
import copy
from tqdm import tqdm
import itertools
from skimage.filters import threshold_otsu
from skimage import measure

In [ ]:
#resolution
px_x10=0.215433 #um/pixel

def count_digit(value):
    return sum(c.isdigit() for c in str(value))

In [ ]:
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
files=os.listdir(npz_path)
radial_file=[i for i in files if "radial_file_result_x40" in i]
anatomy_file=[i for i in files if "anatomical_parameters_x40" in i]

#
npz_radial_file=np.load(os.path.join(npz_path, radial_file[0]), allow_pickle=True)
npz_anatomical_parameter=np.load(os.path.join(npz_path, anatomy_file[0]))

In [ ]:
#get anatomical parameters
cell_area=npz_anatomical_parameter["cell_area_list"]
cell_perimeter=npz_anatomical_parameter["cell_perimeter_list"]
cell_circle_level=npz_anatomical_parameter["cell_circle_level_list"]
lumen_area=npz_anatomical_parameter["lumen_area_list"]
lumen_perimeter=npz_anatomical_parameter["lumen_perimeter_list"]
lumen_circle_level=npz_anatomical_parameter["lumen_circle_level_list"]
cellwall_occupancy=npz_anatomical_parameter["cellwall_occuapncy_list"]
flm=npz_anatomical_parameter["flm"]
#twall_thickness_conv=npz_anatomical_parameter["twall_thickness_list"]

#cell position}
cell_cx_list=npz_anatomical_parameter["cell_cx_list"]
cell_cy_list=npz_anatomical_parameter["cell_cy_list"]
lumen_cx_list=npz_anatomical_parameter["lumen_cx_list"]
lumen_cy_list=npz_anatomical_parameter["lumen_cy_list"]

#inof of radial file assignement
radial_file_result=npz_radial_file["radial_file_result_list"]
radial_file_summary=npz_radial_file["radial_file_summary"]
label_map=npz_radial_file["label_map"]

In [ ]:
#anatomical parameter extractions of each file
radial_file_cell_area=[]
radial_file_cell_perimeter=[]
radial_file_cell_circle_level=[]
radial_file_cell_cx=[]
radial_file_cell_cy=[]
radial_file_lumen_area=[]
radial_file_lumen_perimeter=[]
radial_file_lumen_circle_level=[]
radial_file_lumen_cx=[]
radial_file_lumen_cy=[]
radial_file_cellwall_occupancy=[]
#radial_file_twall_thickness_conv=[]


for i in tqdm(range(len(radial_file_result))):
    radial_file_cell_area_temp=[cell_area[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_cell_perimeter_temp=[cell_perimeter[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_cell_circle_level_temp=[cell_circle_level[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_cell_cx_temp=[cell_cx_list[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_cell_cy_temp=[cell_cy_list[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_lumen_area_temp=[lumen_area[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_lumen_perimeter_temp=[lumen_perimeter[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_lumen_circle_level_temp=[lumen_circle_level[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_lumen_cx_temp=[lumen_cx_list[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_lumen_cy_temp=[lumen_cy_list[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    radial_file_cellwall_occupancy_temp=[cellwall_occupancy[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    #radial_file_twall_thickness_temp=[twall_thickness_conv[radial_file_result[i][j]] for j in range(len(radial_file_result[i]))]
    
    
    
    radial_file_cell_area.append(radial_file_cell_area_temp)
    radial_file_cell_perimeter.append(radial_file_cell_perimeter_temp)
    radial_file_cell_circle_level.append(radial_file_cell_circle_level_temp)
    radial_file_cell_cx.append(radial_file_cell_cx_temp)
    radial_file_cell_cy.append(radial_file_cell_cy_temp)
    radial_file_lumen_area.append(radial_file_lumen_area_temp)
    radial_file_lumen_perimeter.append(radial_file_lumen_perimeter_temp)
    radial_file_lumen_circle_level.append(radial_file_lumen_circle_level_temp)
    radial_file_lumen_cx.append(radial_file_lumen_cx_temp)
    radial_file_lumen_cy.append(radial_file_lumen_cy_temp)
    radial_file_cellwall_occupancy.append(radial_file_cellwall_occupancy_temp)
    #radial_file_twall_thickness_conv.append(radial_file_twall_thickness_temp)
    

In [ ]:
#visualzie all results
save_path=os.path.join(current_path, "fig_save")
radial_file_path=os.path.join(save_path, "radial_file_result")

if os.path.exists(radial_file_path)==False:
    os.makedirs(radial_file_path)

for i in range(len(radial_file_result)):
    flm_copy=copy.deepcopy(flm[:,:])
    height, width = flm_copy.shape
    zero_map=np.zeros((height, width)).flatten()
    for j in range(len(radial_file_result[i])):
        
        ind=np.where(label_map==radial_file_result[i][j])
        zero_map[ind]=255

    zero_map=zero_map.reshape(height, width)
    flm_copy[np.where(zero_map==0)]=0
    plt.figure(figsize=(10, 6))
    plt.imshow(flm_copy)
    plt.tight_layout()
    #plt.savefig(os.path.join(radial_file_path, "radial_file_result_im_"+str(i)+".png"), dpi=600)
    plt.show()
    
    #save
    cv2.imwrite(os.path.join(radial_file_path, "radial_file_result_im_"+str(i)+".png"), flm_copy)

   

In [ ]:
plt.rcParams["font.family"]="Times New Roman"

for i in range(len(radial_file_result)):
    x_ticks=np.linspace(0, 1, len(radial_file_lumen_area[i]))

    fig, ax=plt.subplots(2,2, figsize=(10,6))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10, label="cell area", c="red")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10, c="red", alpha=0.6)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, label="lumen area", c="blue", marker="^")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, c="blue", alpha=0.6)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10
                 -np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, label="cell wall area", c="green", marker="x")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10
                 -np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, c="green")
    ax[0,0].set_ylabel("Area (um^2)")
    ax[0,0].legend(loc=2)

    ax[0,1].scatter(x_ticks, np.asarray(radial_file_cell_perimeter[i])*px_x10, label="cell perimeter", c="red")
    ax[0,1].plot(x_ticks, np.asarray(radial_file_cell_perimeter[i])*px_x10, c="red", alpha=0.6)
    ax[0,1].scatter(x_ticks, np.asarray(radial_file_lumen_perimeter[i])*px_x10, label="lumen perimeter", c="blue", marker="^")
    ax[0,1].plot(x_ticks, np.asarray(radial_file_lumen_perimeter[i])*px_x10, c="blue", alpha=0.6)
    ax[0,1].set_ylabel("Length (um)")
    ax[0,1].legend(loc=2)

    ax[1,0].scatter(x_ticks, np.asarray(radial_file_cell_circle_level[i]), label="cell circle level", c="red")
    ax[1,0].plot(x_ticks, np.asarray(radial_file_cell_circle_level[i]), c="red", alpha=0.6)
    ax[1,0].scatter(x_ticks, np.asarray(radial_file_lumen_circle_level[i]), label="lumen circle level", c="blue", marker="^")
    ax[1,0].plot(x_ticks, np.asarray(radial_file_lumen_circle_level[i]), c="blue", alpha=0.6)
    ax[1,0].set_ylabel("Circle level (a.u.)")
    ax[1,0].legend(loc=2)

    ax[1,1].scatter(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    label="cell wall occupancy", c="red")
    ax[1,1].plot(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    c="red", alpha=0.6)
    ax[1,1].set_ylabel("Occupancy (%)")
    ax[1,1].legend(loc=2)

    plt.tight_layout()
    #plt.savefig(os.path.join(radial_file_path, "radial_file_result_param_count_base_"+str(i)+".png"), dpi=600)

    plt.show()

In [ ]:
#cell centroid position based visualization
plt.rcParams["font.family"]="Times New Roman"

for i in range(len(radial_file_result)):
    x_ticks=np.asarray(radial_file_cell_cx[i])*px_x10

    fig, ax=plt.subplots(2,2, figsize=(10,6))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10, label="cell area", c="red")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10, c="red", alpha=0.6)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, label="lumen area", c="blue", marker="^")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, c="blue", alpha=0.6)
    ax[0,0].scatter(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10
                 -np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, label="cell wall area", c="green", marker="^")
    ax[0,0].plot(x_ticks, np.asarray(radial_file_cell_area[i])*px_x10*px_x10
                 -np.asarray(radial_file_lumen_area[i])*px_x10*px_x10, c="green", alpha=0.6)
    ax[0,0].set_ylabel("Area (um^2)")
    ax[0,0].legend(loc=2)

    ax[0,1].scatter(x_ticks, np.asarray(radial_file_cell_perimeter[i])*px_x10, label="cell perimeter", c="red")
    ax[0,1].plot(x_ticks, np.asarray(radial_file_cell_perimeter[i])*px_x10, c="red", alpha=0.6)
    ax[0,1].scatter(x_ticks, np.asarray(radial_file_lumen_perimeter[i])*px_x10, label="lumen perimeter", c="blue", marker="^")
    ax[0,1].plot(x_ticks, np.asarray(radial_file_lumen_perimeter[i])*px_x10, c="blue", alpha=0.6)
    ax[0,1].set_ylabel("Length (um)")
    ax[0,1].legend(loc=2)

    ax[1,0].scatter(x_ticks, np.asarray(radial_file_cell_circle_level[i]), label="cell circle level", c="red")
    ax[1,0].plot(x_ticks, np.asarray(radial_file_cell_circle_level[i]), c="red", alpha=0.6)
    ax[1,0].scatter(x_ticks, np.asarray(radial_file_lumen_circle_level[i]), label="lumen circle level", c="blue", marker="^")
    ax[1,0].plot(x_ticks, np.asarray(radial_file_lumen_circle_level[i]), c="blue", alpha=0.6)
    ax[1,0].set_ylabel("Circle level (a.u.)")
    ax[1,0].legend(loc=2)

    ax[1,1].scatter(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    label="cell wall occupancy", c="red")
    ax[1,1].plot(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    c="red", alpha=0.6)
    ax[1,1].set_ylabel("Occupancy (%)")
    ax[1,1].legend(loc=2)

    plt.tight_layout()
    #plt.savefig(os.path.join(radial_file_path, "radial_file_result_param_position_base_"+str(i)+".png"), dpi=600)

    plt.show()

In below part, radial file based anatomical parameter extraction is conducted.

In [ ]:
cell_height_list=[]
cell_width_list=[]
lumen_width_list=[]
lumen_height_list=[]
twall_thickness=[]
double_wall_thickness=[]

#define lumen region by otsu binarization
threshold = threshold_otsu(flm)
flm_binary=np.where(flm<threshold, 255, 0)
flm_binary=np.uint8(flm_binary)
flm_binary_inv=cv2.bitwise_not(flm_binary)
label_map_copy=copy.deepcopy(label_map)
criterion_thres=0.5


for i in range(len(radial_file_result)):
    cell_height_list_temp=[]
    cell_width_list_temp=[]
    lumen_width_list_temp=[]
    lumen_height_list_temp=[]
    twall_thickness_temp=[]
    double_wall_thickness_temp=[]
    
    for j in tqdm(range(len(radial_file_result[i]))):
        #get centroid cooridinates of adjacent cell
        if j==0:
            target_ind, adj_2_ind, adj_1_ind = radial_file_result[i][j:j+3]
        elif j==len(radial_file_result[i])-2:
            adj_1_ind, target_ind, adj_2_ind = radial_file_result[i][j-1:]
        elif j==len(radial_file_result[i])-1:
            adj_2_ind, adj_1_ind, target_ind = radial_file_result[i][j-2:]
        else:
            adj_1_ind, target_ind, adj_2_ind = radial_file_result[i][j-1:j+2]
        
        height, width=flm.shape
        
        #binarized map whose values are 255 in target and adjacent cells is created
        binarized_map=np.zeros((height, width)).flatten()
        binarized_map_1=np.zeros((height, width)).flatten()
        binarized_map_2=np.zeros((height, width)).flatten()
        
        #target cell
        binarized_map[np.where(label_map==target_ind)]=255
        binarized_map=binarized_map.reshape((height, width))
        
        #adj cell
        binarized_map_1[np.where(label_map==adj_1_ind)]=255
        binarized_map_2[np.where(label_map==adj_2_ind)]=255
        binarized_map_1=binarized_map_1.reshape((height, width))
        binarized_map_2=binarized_map_2.reshape((height, width))
        
        #adjustment of micro-orientation
        #calculate region props
        props_target=measure.regionprops(np.uint8(binarized_map))
        props_1=measure.regionprops(np.uint8(binarized_map_1))
        props_2=measure.regionprops(np.uint8(binarized_map_2))
        
        #get centorids
        cy_target, cx_target = props_target[0].centroid
        cy_adj_1, cx_adj_1 = props_1[0].centroid
        cy_adj_2, cx_adj_2 = props_2[0].centroid
        
        #convert them to interger
        cy_target, cx_target = int(cy_target), int(cx_target)
        cy_adj_1, cx_adj_1 = int(cy_adj_1), int(cx_adj_1)
        cy_adj_2, cx_adj_2 = int(cy_adj_2), int(cx_adj_2)
        
        #define angle for rotation
        if j==0:
            diff_x = cx_adj_1-cx_target
            diff_y = cy_adj_1-cy_target
            
        elif j==len(radial_file_result[i])-1:
            diff_x = cx_target-cx_adj_2
            diff_y = cy_target-cy_adj_2
            
        else:
            diff_x = cx_adj_2-cx_target
            diff_y = cy_adj_2-cy_target
        
        #calculate angle
        angle=np.arctan2(diff_y, diff_x)*180/np.pi
        
        #rotate the image around the center of gravity of cell area
        scale=1
        center=(cx_target, cy_target)
        angle_rot=-angle
        trans = cv2.getRotationMatrix2D(center, angle_rot , scale)
        
        #affine transformation#calculate angle
        binarized_map_rot = cv2.warpAffine(binarized_map, trans, (width, height), flags=cv2.INTER_NEAREST)
        flm_binary_rot = cv2.warpAffine(flm_binary, trans, (width, height))
        
        
        #################
        # Cell geometry #
        #################
        
        #cell width
        n=0
        criterion=list(binarized_map_rot[cy_target]).count(255)
        
        while(True):
            target_px_1=cy_target+2*n
            target_px_2=cy_target-2*n
            
            target_slice_1=binarized_map_rot[target_px_1]
            target_slice_2=binarized_map_rot[target_px_2]
            
            #if one of two slices only contains zero, break loop
            if (list(target_slice_1).count(255))/criterion < criterion_thres or (list(target_slice_2).count(255))/criterion < criterion_thres:
                break
                
            #add counter & reset criterion
            n+=1
            
        #define scanning region
        scanning_height=int(2*n*0.75)
        scanning_region=binarized_map_rot[cy_target-scanning_height : cy_target+scanning_height].flatten()
        cell_width=list(scanning_region).count(255)/(2*scanning_height)
        
        
        #cell height
        m=0
        criterion=list(binarized_map_rot[:, cx_target]).count(255)
        
        while(True):
            target_px_1=cx_target+2*m
            target_px_2=cx_target-2*m
            
            target_slice_1=binarized_map_rot[:, target_px_1]
            target_slice_2=binarized_map_rot[:, target_px_2]
            
            #if one of two slices only contains zero, break loop
            if (list(target_slice_1).count(255))/criterion < criterion_thres  or (list(target_slice_2).count(255))/criterion < criterion_thres:
                break
                    
            #add counter
            m+=1
            
        #define scanning region
        scanning_width=int(2*m*0.75)
        scanning_region=binarized_map_rot[:, cx_target-scanning_width : cx_target+scanning_width].flatten()
        cell_height=list(scanning_region).count(255)/(2*scanning_width)
        
        
        ##################
        # Lumen geometry #
        ##################
        
        lumen_ind=np.where((binarized_map_rot.flatten()==255) & (flm_binary_rot.flatten()==255))
        
        #especially in latewood, some cells doesnt have lumen area
        if len(lumen_ind[0])==0:
            lumen_width=0
            lumen_height=0
        
        else:
            lumen_binarized_map=np.zeros((height, width)).flatten()
            lumen_binarized_map[lumen_ind]=255
            lumen_binarized_map=lumen_binarized_map.reshape((height, width))

            #calculate centroids of lumen after rotation
            props_lumen=measure.regionprops(np.uint8(lumen_binarized_map))
            lumen_y, lumen_x = props_lumen[0].centroid
            lumen_y, lumen_x = int(lumen_y), int(lumen_x)

            #we assume that centorids of lumen are alsmost the same as those of cell
            #Exactly, we already obtained the data which support above assumption
            #lumen width
            n=0
            criterion=list(lumen_binarized_map[lumen_y]).count(255)

            while(True):
                if criterion==0:
                    break

                target_px_1=lumen_y+2*n
                target_px_2=lumen_y-2*n

                target_slice_1=lumen_binarized_map[target_px_1]
                target_slice_2=lumen_binarized_map[target_px_2]

                #if one of two slices only contains zero, break loop
                if (list(target_slice_1).count(255))/criterion < criterion_thres or (list(target_slice_2).count(255))/criterion < criterion_thres:
                    break

                #add counter
                n+=1

            #define scanning region
            scanning_height=int(2*n*0.75)
            scanning_region=lumen_binarized_map[lumen_y-scanning_height : lumen_y+scanning_height].flatten()

            if scanning_height==0:
                lumen_width=list(scanning_region).count(255)
            else:
                lumen_width=list(scanning_region).count(255)/(2*scanning_height)


            #lumen height
            m=0
            criterion=list(lumen_binarized_map[:, lumen_x]).count(255)

            while(True):
                if criterion==0:
                    break

                target_px_1=lumen_x+2*m
                target_px_2=lumen_x-2*m

                target_slice_1=lumen_binarized_map[:, target_px_1]
                target_slice_2=lumen_binarized_map[:, target_px_2]

                #if one of two slices only contains zero, break loop
                if (list(target_slice_1).count(255))/criterion < criterion_thres or (list(target_slice_2).count(255))/criterion < criterion_thres:
                    break

                #add counter
                m+=1

            #define scanning region
            scanning_width=int(2*m*0.75)
            scanning_region=lumen_binarized_map[:, lumen_x-scanning_width : lumen_x+scanning_width].flatten()
            if scanning_width==0:
                lumen_height=list(scanning_region).count(255)
            else:
                lumen_height=list(scanning_region).count(255)/(2*scanning_width)
        
        
        
        #############################
        # Tangential wall thickness #
        #############################
        
        
        #First, thickness of tangential wall on adj_1 side is calculated
        #create binarized map
        cellwall_binary=np.where(label_map_copy==target_ind, flm_binary_inv.flatten(), 0)
        cellwall_binary=cellwall_binary.reshape((height, width))
        
        
        if j==0:
            pass
        else:
            cell_adj_1_x=cell_cx_list[adj_1_ind]
            cell_adj_1_y=cell_cy_list[adj_1_ind]
            diff_y=cy_target-cell_adj_1_y
            diff_x=cx_target-cell_adj_1_x
            
            #calculate angle
            angle=np.arctan2(diff_y, diff_x)*180/np.pi
            
            #get rotation matrix
            scale=1
            center=(cx_target, cy_target)
            angle_rot=angle
            trans = cv2.getRotationMatrix2D(center, angle_rot , scale)
            #affine transformation
            cellwall_binary_rot_1 = cv2.warpAffine(cellwall_binary, trans, (width,height), flags=cv2.INTER_NEAREST)
            
            #scanning 
            n=0
            criterion=list(cellwall_binary_rot_1[cy_target]).count(255)/2
            twall_thickness_1_list=[]
            
            while(True):
                target_px_1=cy_target+2*n
                target_px_2=cy_target-2*n

                target_slice_1=cellwall_binary_rot_1[target_px_1]
                target_slice_2=cellwall_binary_rot_1[target_px_2]
                
                #max and min ind of cell wall (255)
                cellwall_target_1=np.where(target_slice_1==255)[0]
                cellwall_target_2=np.where(target_slice_2==255)[0]
                
                if len(cellwall_target_1)==0 or len(cellwall_target_2)==0:
                    break
                
                max_ind_1=cellwall_target_1[-1]
                min_ind_1=cellwall_target_1[0]
                max_ind_2=cellwall_target_2[-1]
                min_ind_2=cellwall_target_2[0]
                
                #count cell wall pixels
                #if lumen doesnt exist, cell wall thickness is calculated by assuming that two tangential walls have same thickness
                #target_px_1
                if len(target_slice_1[min_ind_1:max_ind_1])==list(target_slice_1[min_ind_1:max_ind_1]).count(255):
                    result_1=len(target_slice_1[min_ind_1:max_ind_1])/2
                else:
                    lumen_ind=np.where(target_slice_1[min_ind_1:max_ind_1]==0)[0]
                    result_1=list(target_slice_1[min_ind_1:min_ind_1+lumen_ind[-1]]).count(255)
                
                if result_1==0:
                    break
                
                #target_px_2
                if len(target_slice_2[min_ind_2:max_ind_2])==list(target_slice_2[min_ind_2:max_ind_2]).count(255):
                    result_2=len(target_slice_2[min_ind_2:max_ind_2])/2
                else:
                    lumen_ind=np.where(target_slice_2[min_ind_2:max_ind_2]==0)[0]
                    result_2=list(target_slice_2[min_ind_2:min_ind_2+lumen_ind[-1]]).count(255)
                
                if result_2==0:
                    break
                
                #if one of two slices only contains zero, break loop
                if criterion/result_1 < criterion_thres or criterion/result_2 < criterion_thres:
                    break
                
                #save result
                twall_thickness_1_list.append((result_1+result_2)/2)

                #add counter
                n+=1
            
            #define scanning region
            twall_thickness_1_list=np.asarray(twall_thickness_1_list)
            twall_1=np.mean(twall_thickness_1_list)
            
            #double wall thickness is calculated by summing wall thickness of adj_1 side of present cell and
            #that of adj_2 side of previous cell
            double_wall=twall_1+twall_2_former
            
        
        #adj_2
        
        if j==len(radial_file_result[i])-1:
            pass
        else:
            cell_adj_2_x=cell_cx_list[adj_2_ind]
            cell_adj_2_y=cell_cy_list[adj_2_ind]
            diff_y=cell_adj_2_y-cy_target
            diff_x=cell_adj_2_x-cx_target
            
            #calculate angle
            angle=np.arctan2(diff_y, diff_x)*180/np.pi
            
            #get rotation matrix
            scale=1
            center=(cx_target, cy_target)
            angle_rot=angle
            trans = cv2.getRotationMatrix2D(center, angle_rot , scale)
            #affine transformation
            cellwall_binary_rot_2 = cv2.warpAffine(cellwall_binary, trans, (width,height), flags=cv2.INTER_NEAREST)
        
            #scanning 
            m=0
            criterion=list(cellwall_binary_rot_2[cy_target]).count(255)/2
            twall_thickness_2_list=[]
            double_wall_thickness_list=[]
            
            while(True):
                target_px_1=cy_target+2*m
                target_px_2=cy_target-2*m
                
                
                ###single wall thickness calculation###
                target_slice_1=cellwall_binary_rot_2[target_px_1]
                target_slice_2=cellwall_binary_rot_2[target_px_2]
                
                #max and min ind of cell wall (255)
                cellwall_target_1=np.where(target_slice_1==255)[0]
                cellwall_target_2=np.where(target_slice_2==255)[0]
                
                if len(cellwall_target_1)==0 or len(cellwall_target_2)==0:
                    break
                
                max_ind_1=cellwall_target_1[-1]
                min_ind_1=cellwall_target_1[0]
                max_ind_2=cellwall_target_2[-1]
                min_ind_2=cellwall_target_2[0]
                
                #count cell wall pixels
                #if lumen doesnt exist, cell wall thickness is calculated by assuming that two tangential walls have same thickness
                #target_px_1
                if len(target_slice_1[min_ind_1:max_ind_1])==list(target_slice_1[min_ind_1:max_ind_1]).count(255):
                    result_1=len(target_slice_1[min_ind_1:max_ind_1])/2
                else:
                    lumen_ind=np.where(target_slice_1[min_ind_1:max_ind_1]==0)[0]
                    result_1=list(target_slice_1[min_ind_1+lumen_ind[0] : max_ind_1]).count(255)
                
                if result_1==0:
                    break
                
                #target_px_2
                if len(target_slice_2[min_ind_2:max_ind_2])==list(target_slice_2[min_ind_2:max_ind_2]).count(255):
                    result_2=len(target_slice_2[min_ind_2:max_ind_2])/2
                else:
                    lumen_ind=np.where(target_slice_2[min_ind_2:max_ind_2]==0)[0]
                    result_2=list(target_slice_2[min_ind_2+lumen_ind[0] : max_ind_2]).count(255)
                
                if result_2==0:
                    break
                
                #if one of two slices only contains zero, break loop
                if criterion/result_1 < criterion_thres or criterion/result_2 < criterion_thres:
                    break
                
                #save result
                twall_thickness_2_list.append((result_1+result_2)/2)

                #add counter
                m+=1
            
            #define scanning region
            twall_thickness_2_list=np.asarray(twall_thickness_2_list)
            twall_2=np.mean(twall_thickness_2_list)
            

        #save results of each file
        cell_height_list_temp.append(cell_height)
        cell_width_list_temp.append(cell_width)
        lumen_width_list_temp.append(lumen_width)
        lumen_height_list_temp.append(lumen_height)
        
        if j==0:
            twall_thickness_temp.append(twall_2)
        elif j==len(radial_file_result[i])-1:
            twall_thickness_temp.append(twall_1)
        else:
            twall_thickness_temp.append((twall_1+twall_2)/2)
            
        #set former twall thickness
        twall_2_former=twall_2
        if j>0:
            double_wall_thickness_temp.append(double_wall)
    
    #save all results 
    cell_height_list.append(cell_height_list_temp)
    cell_width_list.append(cell_width_list_temp)
    lumen_width_list.append(lumen_width_list_temp)
    lumen_height_list.append(lumen_height_list_temp)
    twall_thickness.append(twall_thickness_temp)
    double_wall_thickness.append(double_wall_thickness_temp)
        

In [ ]:
#calculate Morks index
Mork_formula_1=[]
Mork_formula_2=[]

#some cells dont have lumen width, therefore calculated values are like L/2M (L: lumen width, M: cell wall thickness)
for i in tqdm(range(len(lumen_width_list))):
    Mork_1=[(lumen_width_list[i][j+1])/(2*double_wall_thickness[i][j]) for j in range(len(lumen_width_list[i])-1)]
    Mork_2=[(lumen_width_list[i][j])/(2*twall_thickness[i][j]) for j in range(len(lumen_width_list[i]))]
    Mork_formula_1.append(Mork_1)
    Mork_formula_2.append(Mork_2)

In [ ]:
#save result
np.savez(os.path.join(npz_path, "radial_file_based_anatomical_parameter_x40.npz"),
         cell_area=radial_file_cell_area, cell_perimeter=radial_file_cell_perimeter,
         cell_circle_level=radial_file_cell_circle_level, 
         cell_cx=radial_file_cell_cx, cell_cy=radial_file_cell_cy, 
         lumen_area=radial_file_lumen_area, lumen_perimeter=radial_file_lumen_perimeter,
         lumen_circle_level=radial_file_lumen_circle_level,
         lumen_cx=radial_file_lumen_cx, lumen_cy=radial_file_lumen_cy,
         cellwall_occupancy=radial_file_cellwall_occupancy,
         cell_height=cell_height_list, cell_width=cell_width_list, lumen_height=lumen_height_list,
         lumen_width=lumen_width_list,
         twall_thickness=twall_thickness, double_wall_thickness=double_wall_thickness,
         Mork_index_1=Mork_formula_1, Mork_index_2=Mork_formula_2,
         flm=flm, label_map=label_map, radial_file_result=radial_file_result)

In [ ]:
#cell centroid position based visualization
plt.rcParams["font.family"]="Times New Roman"

for i in range(len(radial_file_result)):
    x_ticks=np.asarray(radial_file_cell_cx[i])*px_x10

    fig, ax=plt.subplots(2,2, figsize=(10,6))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    #cell & lumen dimension
    #cell height
    ax[0,0].scatter(x_ticks, np.asarray(cell_height_list[i])*px_x10, label="cell height", c="red")
    ax[0,0].plot(x_ticks, np.asarray(cell_height_list[i])*px_x10, c="red", alpha=0.6)
    #cell width
    ax[0,0].scatter(x_ticks, np.asarray(cell_width_list[i])*px_x10, label="cell width", c="blue", marker="^")
    ax[0,0].plot(x_ticks, np.asarray(cell_width_list[i])*px_x10, c="blue", alpha=0.6)
    #lumen height
    ax[0,0].scatter(x_ticks, np.asarray(lumen_height_list[i])*px_x10, label="lumen height", c="green")
    ax[0,0].plot(x_ticks, np.asarray(lumen_height_list[i])*px_x10, c="green", alpha=0.6)
    #lumen width
    ax[0,0].scatter(x_ticks, np.asarray(lumen_width_list[i])*px_x10, label="lumen width", c="purple", marker="^")
    ax[0,0].plot(x_ticks, np.asarray(lumen_width_list[i])*px_x10, c="purple", alpha=0.6)
    
    #twall thickness
    ax[0,0].set_ylabel("diameter (um)")
    ax[0,0].legend(loc=4)

    ax[0,1].scatter(x_ticks, np.asarray(twall_thickness[i])*px_x10,  c="red")
    ax[0,1].plot(x_ticks, np.asarray(twall_thickness[i])*px_x10, c="red", alpha=0.6)
    ax[0,1].set_ylabel("thickness (um)")
    #ax[0,1].legend(loc=2)
    
    #Mork index
    ax[1,0].scatter(x_ticks[1:], np.asarray(Mork_formula_1[i]), label="Mork's index (formula 1)", c="red")
    ax[1,0].plot(x_ticks[1:], np.asarray(Mork_formula_1[i]), c="red", alpha=0.6)
    ax[1,0].scatter(x_ticks[1:], np.asarray(Mork_formula_2[i][1:]), label="Mork's index (formula 2)", c="blue", marker="^")
    ax[1,0].plot(x_ticks[1:], np.asarray(Mork_formula_2[i][1:]), c="blue", alpha=0.6)
    ax[1,0].set_ylabel("Mork's index (a.u.)")
    ax[1,0].legend(loc=4)

    ax[1,1].scatter(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    label="cell wall occupancy", c="red")
    ax[1,1].plot(x_ticks, (1-np.asarray(radial_file_lumen_area[i])/np.asarray(radial_file_cell_area[i]))*100, 
                    c="red", alpha=0.6)
    ax[1,1].set_ylabel("Occupancy (%)")
    #ax[1,1].legend(loc=2)

    plt.tight_layout()
    #plt.savefig(os.path.join(radial_file_path, "radial_file_result_param_position_base_"+str(i)+".png"), dpi=600)

    plt.show()